In [71]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
import glob
import math
import chardet
from datetime import datetime

def download_excel():
    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 브라우저를 띄우지 않도록 설정
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # ChromeDriver의 경로 설정 및 생성
    driver_path = ChromeDriverManager().install()
    correct_driver_path = os.path.join(os.path.dirname(driver_path), "chromedriver.exe")
    driver = webdriver.Chrome(service=Service(executable_path=correct_driver_path, options=chrome_options))

    try:
        # URL로 이동
        driver.get('https://mtis.komsa.or.kr/traffic/liveSea')
        time.sleep(5)  # 페이지 로딩 대기

        # 왼쪽 하단의 '결과 조회' 버튼 클릭
        results_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="pannel02_02"]/div/div[2]/ul/li/button[2]'))
        )
        results_button.click()

        time.sleep(20)  # 결과 조회 대기

        # 오른쪽 상단의 OPEN 버튼 클릭
        open_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'right_btn_con'))
        )
        open_button.click()

        time.sleep(5)  # 다운로드 대기

        # 엑셀 다운로드 버튼 클릭
        excel_download_button = driver.find_element(By.XPATH, '//*[@id="rightSide"]/div/div[2]/div')
        driver.execute_script("arguments[0].scrollIntoView(true);", excel_download_button)
        driver.execute_script("arguments[0].click();", excel_download_button)

        time.sleep(10)  # 다운로드 대기
        print("엑셀 파일 다운로드 완료.")

    except Exception as e:
        print(f"Error during scraping or processing: {type(e).__name__}: {str(e)}")
    
    finally:
        driver.quit()  # 드라이버 종료

    # 다운로드한 파일의 경로를 찾기
    download_folder = os.path.expanduser(r"~/Downloads")  # 다운로드 폴더 경로
    excel_files = glob.glob(os.path.join(download_folder, "*.xlsx"))  # 다운로드 폴더 내의 모든 엑셀 파일
    latest_file = max(excel_files, key=os.path.getctime)  # 가장 최근에 다운로드된 엑셀 파일
    print(f"다운로드한 엑셀 파일 경로: {latest_file}")

    return latest_file  # 최신 엑셀 파일 경로 반환

# 나머지 데이터 처리 및 거리 계산 함수 (이전 코드와 동일)

# 해리 (nautical mile)을 km로 변환
nautical_mile_to_km = 1.852

# 두 좌표 간의 거리 계산 함수 (Haversine formula)
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # 지구의 반지름 (km)
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # 거리 (km)

# 주어진 좌표가 반지름 내에 있는지 확인하는 함수
def is_within_radius(lat1, lon1, lat2, lon2, radius_nm):
    radius_km = radius_nm * nautical_mile_to_km  # 반지름을 km로 변환
    distance = haversine(lon1, lat1, lon2, lat2)
    return distance <= radius_km

# 좌표가 주어진 경계(선) 내에 있는지 확인하는 함수
def is_within_area(lat, lon, start_lat, start_lon, end_lat, end_lon):
    return min(start_lat, end_lat) <= lat <= max(start_lat, end_lat) and min(start_lon, end_lon) <= lon <= max(start_lon, end_lon)

# 범위에 대한 좌표 설정
orukdo_lat, orukdo_lon = 35 + 5/60 + 28/3600, 129 + 7/60 + 36/3600
orukdo_end_lat, orukdo_end_lon = 35 + 20/60 + 21/3600, 129 + 24/60 + 1.5/3600
saengdo_lat, saengdo_lon = 35 + 2/60 + 13.7/3600, 129 + 5/60 + 34.8/3600
saengdo_end_lat, saengdo_end_lon = 34 + 45/60 + 30/3600, 128 + 52/60 + 5/3600

gadeokdo_lat, gadeokdo_lon = 34 + 59/60 + 22.2/3600, 128 + 49/60 + 44.6/3600
gadeokdo_end_lat1, gadeokdo_end_lon1 = 35 + 3/60 + 59/3600, 129 + 13/60 + 17/3600
gadeokdo_end_lat2, gadeokdo_end_lon2 = 34 + 43/60 + 9/3600, 128 + 36/60 + 15/3600

palmi_lat, palmi_lon = 37 + 21/60 + 29.8/3600, 126 + 30/60 + 38.9/3600
radius_nm = 20  # 반지름 20해리

# 범위 확인 함수
def check_in_first_area(lat, lon):
    return (is_within_area(lat, lon, orukdo_lat, orukdo_lon, orukdo_end_lat, orukdo_end_lon) or
            is_within_radius(lat, lon, orukdo_lat, orukdo_lon, radius_nm) or
            is_within_area(lat, lon, saengdo_lat, saengdo_lon, saengdo_end_lat, saengdo_end_lon) or
            is_within_radius(lat, lon, saengdo_lat, saengdo_lon, radius_nm))

def check_in_second_area(lat, lon):
    return (is_within_area(lat, lon, gadeokdo_lat, gadeokdo_lon, gadeokdo_end_lat1, gadeokdo_end_lon1) or
            is_within_area(lat, lon, gadeokdo_lat, gadeokdo_lon, gadeokdo_end_lat2, gadeokdo_end_lon2) or
            is_within_radius(lat, lon, gadeokdo_lat, gadeokdo_lon, radius_nm))

def check_in_third_area(lat, lon):
    return is_within_radius(lat, lon, palmi_lat, palmi_lon, radius_nm)

def determine_area(lat, lon):
    in_first_area = check_in_first_area(lat, lon)
    in_second_area = check_in_second_area(lat, lon)
    
    if in_first_area and in_second_area:
        return 4  # 겹치는 경우
    elif in_first_area:
        return 1
    elif in_second_area:
        return 2
    elif check_in_third_area(lat, lon):
        return 3
    else:
        return 0

def convert_excel_to_csv(excel_path):
    # Excel 파일을 읽어서 CSV로 변환
    csv_path = excel_path.replace('.xlsx', '.csv')  # CSV 파일의 경로 설정
    df = pd.read_excel(excel_path)  # Excel 파일 읽기
    df.to_csv(csv_path, index=False)  # CSV로 저장
    return csv_path

def process_data(csv_file_path):
    # CSV 파일을 읽기
    with open(csv_file_path, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")

    # 인코딩을 사용하여 CSV 파일을 읽기
    df = pd.read_csv(csv_file_path, encoding=encoding)

    # "위경도" 열을 분리하여 "위도"와 "경도" 열 추가
    df[['위도', '경도']] = df['위경도'].str.split(',', expand=True)
    df['위도'] = df['위도'].astype(float)
    df['경도'] = df['경도'].astype(float)
    
    # 여기에 범위 계산 및 기타 처리 코드 삽입
    df['범위'] = df.apply(lambda row: determine_area(row['위도'], row['경도']), axis=1)  # 범위 계산
    
    # loc_type을 1로 설정
    df['loc_type'] = 1  # 기본값 설정

     # 'location' 열 추가: {lat: , lng: } 형식으로 변환
    df['위도'] = df['위도'].astype(str)
    df['경도'] = df['경도'].astype(str)
    df['location'] = df.apply(lambda row: f"{{lat:{row['위도'].strip()}, lng:{row['경도'].strip()}}}", axis=1)

    # 범위가 0인 데이터 제외
    df_filtered = df[df['범위'] != 0]
    df_filtered = df_filtered[df_filtered['교통량(척)'] >= 60]
    
    return df_filtered

# MySQL에 삽입
def insert_into_mysql(df_filtered):
    # MySQL 연결 설정
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="project",
        connect_timeout=28800
    )

    try:
        # 커서 생성
        cursor = connection.cursor()

        # 현재 시간 가져오기
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # 1. 기존 데이터의 end_date를 현재 시간으로 업데이트
        update_end_date_sql = f"""
            UPDATE PortInfoB 
            SET end_date = '{current_time}' 
            WHERE end_date IS NULL;
        """
        cursor.execute(update_end_date_sql)
        connection.commit()  # 트랜잭션 커밋

        # 2. 새로운 데이터 삽입
        for index, row in df_filtered.iterrows():
            loc_type = row.get('loc_type', 1)  # loc_type 기본값 설정

            if row['범위'] == 4:
                # 범위가 4인 경우 KRPUS와 KRBNP에 각각 삽입
                sql_krpus = f"""
                INSERT INTO PortInfoB (port_code, loc_type, location) 
                VALUES ('KRPUS', {loc_type}, '{row['location']}')
                ON DUPLICATE KEY UPDATE 
                    location = VALUES(location);
                """
                cursor.execute(sql_krpus)

                sql_krbnp = f"""
                INSERT INTO PortInfoB (port_code, loc_type, location) 
                VALUES ('KRBNP', {loc_type}, '{row['location']}')
                ON DUPLICATE KEY UPDATE 
                    location = VALUES(location);
                """
                cursor.execute(sql_krbnp)
            else:
                # 범위가 1, 2, 3인 경우 하나의 포트에만 삽입
                port_code = 'KRPUS' if row['범위'] == 1 else 'KRBNP' if row['범위'] == 2 else 'KNINC'
                sql = f"""
                INSERT INTO PortInfoB (port_code, loc_type, location) 
                VALUES ('{port_code}', {loc_type}, '{row['location']}')
                ON DUPLICATE KEY UPDATE 
                    location = VALUES(location);
                """
                cursor.execute(sql)

        connection.commit()  # 트랜잭션 커밋
        print("데이터가 MySQL에 성공적으로 삽입되었습니다.")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        connection.rollback()  # 오류 발생 시 롤백
    finally:
        cursor.close()  # 커서 종료
        connection.close()  # 연결 종료

if __name__ == "__main__":
    excel_path = download_excel()  # 엑셀 다운로드
    csv_path = convert_excel_to_csv(excel_path)  # Excel을 CSV로 변환
    df_filtered = process_data(csv_path)  # 데이터 처리 및 범위 확인
    insert_into_mysql(df_filtered)  # MySQL에 데이터 삽입

Error during scraping or processing: ElementClickInterceptedException: Message: element click intercepted: Element <div id="right_btn_con" class="right_btn_databoard"></div> is not clickable at point (1022, 347). Other element would receive the click: <div class="k-loading-image"></div>
  (Session info: chrome=129.0.6668.101)
Stacktrace:
	GetHandleVerifier [0x00345523+24195]
	(No symbol) [0x002DAA04]
	(No symbol) [0x001D2093]
	(No symbol) [0x0021CE37]
	(No symbol) [0x0021B239]
	(No symbol) [0x0021911B]
	(No symbol) [0x00218724]
	(No symbol) [0x0020D4FD]
	(No symbol) [0x0023AB3C]
	(No symbol) [0x0020CFB4]
	(No symbol) [0x0023ADD4]
	(No symbol) [0x00255280]
	(No symbol) [0x0023A8D6]
	(No symbol) [0x0020BA27]
	(No symbol) [0x0020C43D]
	GetHandleVerifier [0x0060CE13+2938739]
	GetHandleVerifier [0x0065EC69+3274185]
	GetHandleVerifier [0x003D09C2+594722]
	GetHandleVerifier [0x003D7EDC+624700]
	(No symbol) [0x002E37CD]
	(No symbol) [0x002E0528]
	(No symbol) [0x002E06C5]
	(No symbol) [0x002D2C

In [70]:
df_filtered = df_filtered[df_filtered['교통량(척)'] >= 60]  # 교통량 필터링
print(df_filtered)  # 필터링된 데이터 출력

           격자번호              위경도  교통량(척)  밀집도(%)  어선  여객선  화물선  유조선  예인선  \
5   GR3_G3B33_L  35.125, 129.075     115   55.73   1    1   10   39   14   
6   GR3_G3B33_P  35.075, 129.025     110   21.18  35    0    5   10    8   
9   GR3_G3A44_T  35.075, 128.975     100   16.23  60    1    6    1    6   
11  GR3_F2K41_B  37.475, 126.575      95    3.98  27    6    0    1    7   
12  GR3_F2K41_C  37.475, 126.625      91   63.37  30    5    6    7    2   
17  GR3_G3A43_N  35.125, 128.675      76    2.43  63    1    2    1    0   
19  GR3_G3A44_P  35.075, 128.775      72   11.53  25    0    2    0   19   
22  GR3_G3E21_O  34.875, 128.725      70   26.57  20    0    4    3   12   
27  GR3_G3A43_T  35.075, 128.725      66    1.10  59    0    0    0    0   
29  GR3_F2K32_Y  37.275, 126.475      63    2.21  47    0    0    0    0   
31  GR3_G3A43_R  35.075, 128.625      61    0.61  56    0    0    0    0   
32  GR3_G3B33_Q  35.075, 129.075      60   12.31  19    1    7    3    1   

    수상레저기구 